In [1]:
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq

In [2]:
import torch
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer

# ✅ Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Load BART model & tokenizer (Use cached model)
MODEL_NAME = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)


# ✅ Load only first 10 rows (Faster)
train_df = pd.read_csv("/kaggle/input/intentconan2/train.csv")
val_df = pd.read_csv("/kaggle/input/intentconan2/validation.csv")
test_df = pd.read_csv("/kaggle/input/intentconan2/test.csv")

print("Train Dataset Sample:\n", train_df.head())


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Train Dataset Sample:
                                           hatespeech       csType  \
0  Maybe the UN could talk to those asian and afr...  Informative   
1  Maybe the UN could talk to those asian and afr...  Questioning   
2  Maybe the UN could talk to those asian and afr...   Denouncing   
3  Maybe the UN could talk to those asian and afr...     Positive   
4  Juice Jews are worse that nukes. Caption to a ...  Informative   

                                       counterspeech Suggest  Relevance  \
0  The us is the second most polluting country in...       3        4.0   
1  Doesn't everyone on the planet have a responsi...                3.0   
2  The world would be a better place if people we...       1        1.0   
3  You're right, ocean pollution is one of the ma...       3        4.0   
4  Anti-semitism is a serious problem that we nee...       2        3.0   

   Aggressive  Complexity  Comments source  \
0         2.0         3.0       NaN  Human   
1         2.0      

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [4]:
def preprocess_function(examples):
    inputs = [f"Hate Speech: {hs} Intent: {intent}" for hs, intent in zip(examples["hatespeech"], examples["csType"])]
    targets = examples["counterspeech"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=250, truncation=True, padding="max_length")

    # Replace padding token ids with -100 for loss function
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# ✅ Convert Pandas to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# ✅ Tokenize dataset
tokenized_datasets = DatasetDict({
    "train": train_dataset.map(preprocess_function, batched=True),
    "validation": val_dataset.map(preprocess_function, batched=True),
    "test": test_dataset.map(preprocess_function, batched=True),
})


Map:   0%|          | 0/9532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/2971 [00:00<?, ? examples/s]

In [5]:

training_args = TrainingArguments(
    output_dir="./bart_finetuned_samples",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,  # Increase epochs
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100, 
    report_to="none",
    fp16=torch.cuda.is_available(),
    save_total_limit=1,  # Keep only last checkpoint
)


# ✅ Data Collator for Padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()

# ✅ Save Model
trainer.save_model("./bart_finetuned_model_samples")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-0241d28ef573>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.774300,1.878842
2,1.402800,1.839502
3,1.121200,1.834578


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [7]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline

# ✅ Load Fine-Tuned BART Model & Tokenizer
MODEL_PATH = "./bart_finetuned_model_samples"
tokenizer = BartTokenizer.from_pretrained(MODEL_PATH)
model = BartForConditionalGeneration.from_pretrained(MODEL_PATH).to("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load test dataset (10 samples) & Reset Index
test_df = pd.read_csv("/kaggle/input/intentconan2/test.csv")
test_df["BART_counterspeech"] = ""  # Add empty column for results

# ✅ Use the same loaded model for text generation (Avoid reloading)
bart_generator = pipeline(
    "text2text-generation",
    model=model,  # ✅ Directly use the loaded model
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)


def generate_counterspeech_bart(hate_speech, intent):
    """Generates counterspeech using fine-tuned BART"""
    
    # ✅ Use a structured prompt to improve generation quality
    prompt = f"""Generate a {intent} counterspeech response for the following hate speech:

    Hate Speech: {hate_speech}

    Expected Response ({intent} intent):"""

    try:
        response = bart_generator(
            prompt, 
            max_length=100,  
            num_return_sequences=1, 
            temperature=0.7, 
            top_p=0.9, 
            repetition_penalty=1.0, 
            do_sample=True
        )
        
        return response[0]['generated_text'].strip() if response else "No response generated."
    
    except Exception as e:
        print("Error generating response with BART:", e)
        return ""


# ✅ Generate Counterspeech for 10 Samples
print("\nGenerating counterspeech using BART for...")

for i in tqdm(range(len(test_df)), desc="Processing Hate Speech"):
    test_df.at[i, "BART_counterspeech"] = generate_counterspeech_bart(
        test_df.at[i, "hatespeech"], test_df.at[i, "csType"]
    )

# ✅ Print the Generated Counterspeech Samples
print("\nGenerated Counterspeech Samples:")
print(test_df[["hatespeech", "csType", "BART_counterspeech"]])

# ✅ Save Results to CSV
output_file = "/kaggle/working/test_counterspeech.csv"
test_df.to_csv(output_file, index=False)

print(f"\nCounterspeech generation complete! Results saved to {output_file}")


/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Device set to use cuda:0



Generating counterspeech using BART for 10 samples...


Processing Hate Speech: 100%|██████████| 2971/2971 [50:44<00:00,  1.02s/it]


Generated Counterspeech Samples:
                                             hatespeech       csType  \
0     Women are basically childlike, they remain thi...  Informative   
1     Women are basically childlike, they remain thi...   Denouncing   
2     Women are basically childlike, they remain thi...     Positive   
3     Minorities vote to strip wealth and power from...  Informative   
4     Minorities vote to strip wealth and power from...  Questioning   
...                                                 ...          ...   
2966  The Holocaust never happened. It was a coverup...  Informative   
2967  We cannot coexist with millions of Muslims in ...     Positive   
2968  Women are basically childlike, they remain thi...  Informative   
2969  Women do not have equal rights, they are subhu...  Questioning   
2970  Women have no place in our society, its a mens...  Questioning   

                                     BART_counterspeech  
0     I understand that you may have some s